In [ ]:
pip install simpy

1.	Simulate vessels arriving to the container terminal. The time between vessel arrivals follows an exponential distribution with an average of 5 hours (https://docs.python.org/3/library/random.html#random.expovariate). This is the input that drives the simulation. Each vessel carries 150 containers that needs to be discharged (unloaded).**bold text**

In [ ]:
import simpy
import random


def vessel_arrival(env):
    vessel_id = 0  # Unique identifier for each vessel
    while True:
        # Generate the next arrival time
        inter_arrival = random.expovariate(1/5)  # 5 hours mean time
        yield env.timeout(inter_arrival)

        vessel_id += 1
        print(f'Vessel {vessel_id} arrives at {env.now:.2f} hours')
        env.process(discharge_vessel(env, vessel_id))

def discharge_vessel(env, vessel_id):
    num_containers = 150  # Number of containers to discharge
    print(f'Vessel {vessel_id} starts discharging at {env.now:.2f} hours')

    for _ in range(num_containers):
        yield env.timeout(0.1)  # Assuming it takes 0.1 hours to unload one container

    print(f'Vessel {vessel_id} finished discharging at {env.now:.2f} hours')

# Create the SimPy environment
env = simpy.Environment()
env.process(vessel_arrival(env))

# Run the simulation for a specified amount of time
env.run(until=100)  # Run for 100 hours




Vessel 1 arrives at 0.35 hours
Vessel 1 starts discharging at 0.35 hours
Vessel 2 arrives at 0.98 hours
Vessel 2 starts discharging at 0.98 hours
Vessel 3 arrives at 1.52 hours
Vessel 3 starts discharging at 1.52 hours
Vessel 4 arrives at 12.89 hours
Vessel 4 starts discharging at 12.89 hours
Vessel 5 arrives at 13.04 hours
Vessel 5 starts discharging at 13.04 hours
Vessel 6 arrives at 13.44 hours
Vessel 6 starts discharging at 13.44 hours
Vessel 7 arrives at 14.81 hours
Vessel 7 starts discharging at 14.81 hours
Vessel 1 finished discharging at 15.35 hours
Vessel 2 finished discharging at 15.98 hours
Vessel 3 finished discharging at 16.52 hours
Vessel 8 arrives at 18.65 hours
Vessel 8 starts discharging at 18.65 hours
Vessel 9 arrives at 19.95 hours
Vessel 9 starts discharging at 19.95 hours
Vessel 10 arrives at 21.01 hours
Vessel 10 starts discharging at 21.01 hours
Vessel 11 arrives at 23.48 hours
Vessel 11 starts discharging at 23.48 hours
Vessel 4 finished discharging at 27.89 hou

2.	The vessels will need to berth at the terminal and there are only 2 available slots (berth_1 & berth_2). This means that if a vessel arrives and both berths are already occupied, the new vessel will be blocked and has to wait.**bold text**

In [ ]:
!pip install simpy

import simpy
import random

# Constants
NUM_BERTHS = 2  # Number of berths
SIMULATION_TIME = 100  # Simulation time in minutes
ARRIVAL_INTERVAL = 5  # Average interval between vessel arrivals
SERVICE_TIME_MIN = 10  # Minimum service time at the berth
SERVICE_TIME_MAX = 20  # Maximum service time at the berth

def vessel(env, name, berths):
    """
    Vessel process function.
    Args:
    - env: The SimPy environment.
    - name: The name of the vessel.
    - berths: The resource representing the berths.
    """
    print(f"{name} arriving at {env.now:.2f}")
    with berths.request() as request:
        yield request
        print(f"{name} starting service at {env.now:.2f}")
        service_duration = random.randint(SERVICE_TIME_MIN, SERVICE_TIME_MAX)
        yield env.timeout(service_duration)
        print(f"{name} leaving at {env.now:.2f}")

def vessel_generator(env, berths):
    """
    Vessel generator function to create new vessels.
    Args:
    - env: The SimPy environment.
    - berths: The resource representing the berths.
    """
    vessel_count = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        vessel_count += 1
        env.process(vessel(env, f"Vessel {vessel_count}", berths))

# Create the SimPy environment
env = simpy.Environment()

# Create the berths resource with NUM_BERTHS slots
berths = simpy.Resource(env, NUM_BERTHS)

# Start the vessel generator process
env.process(vessel_generator(env, berths))

# Run the simulation
env.run(until=SIMULATION_TIME)


3.	Once the vessel berths, a quay crane (https://en.wikipedia.org/wiki/Container_crane) will start lifting the containers from the vessel to terminal trucks. There are 2 quay cranes and any quay crane can operate on any berth The vessel will use 1 crane only It takes the crane 3 minutes to move one container. The crane must put the container on a truck. If no truck is available, the crane will have to wait until a truck is free before it can start its next move. The crane can use any of the free trucks, it does not have to wait for the same truck to come back again. This means that if only one vessel is in berth, it will never be blocked by waiting for a truck. But if two vessels are in beth, the cranes will sometimes be blocked while waiting for a free truck.**bold text**

In [ ]:
!pip install simpy

import simpy
import random

# Constants
NUM_BERTHS = 2  # Number of berths
NUM_CRANES = 2  # Number of quay cranes
NUM_TRUCKS = 3  # Number of terminal trucks
SIMULATION_TIME = 100  # Simulation time in minutes
ARRIVAL_INTERVAL = 5  # Average interval between vessel arrivals
SERVICE_TIME_MIN = 10  # Minimum service time at the berth
SERVICE_TIME_MAX = 20  # Maximum service time at the berth
CONTAINER_MOVE_TIME = 3  # Time for crane to move one container
NUM_CONTAINERS_MIN = 5  # Minimum number of containers per vessel
NUM_CONTAINERS_MAX = 15  # Maximum number of containers per vessel

def vessel(env, name, berths, cranes, trucks):
    """
    Vessel process function.
    Args:
    - env: The SimPy environment.
    - name: The name of the vessel.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{name} arriving at {env.now:.2f}")
    with berths.request() as berth_request:
        yield berth_request
        print(f"{name} starting service at {env.now:.2f}")

        # Request a quay crane
        with cranes.request() as crane_request:
            yield crane_request
            num_containers = random.randint(NUM_CONTAINERS_MIN, NUM_CONTAINERS_MAX)
            print(f"{name} will move {num_containers} containers")

            for i in range(num_containers):
                with trucks.request() as truck_request:
                    yield truck_request
                    print(f"{name} using crane at {env.now:.2f} to move container {i+1}")
                    yield env.timeout(CONTAINER_MOVE_TIME)

        service_duration = random.randint(SERVICE_TIME_MIN, SERVICE_TIME_MAX)
        yield env.timeout(service_duration)
        print(f"{name} leaving at {env.now:.2f}")

def vessel_generator(env, berths, cranes, trucks):
    """
    Vessel generator function to create new vessels.
    Args:
    - env: The SimPy environment.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    vessel_count = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        vessel_count += 1
        env.process(vessel(env, f"Vessel {vessel_count}", berths, cranes, trucks))

# Create the SimPy environment
env = simpy.Environment()

# Create the berths, cranes, and trucks resources
berths = simpy.Resource(env, NUM_BERTHS)
cranes = simpy.Resource(env, NUM_CRANES)
trucks = simpy.Resource(env, NUM_TRUCKS)

# Start the vessel generator process
env.process(vessel_generator(env, berths, cranes, trucks))

# Run the simulation
env.run(until=SIMULATION_TIME)






4.	The terminal has 3 trucks, transporting containers from the quay cranes to the yard blocks (https://en.wikipedia.org/wiki/Terminal_tractor). It takes the truck 6 minutes to drop off the container at the yard block and come back **again**

In [2]:



!pip install simpy

import simpy
import random



# Constants
NUM_BERTHS = 2  # Number of berths
NUM_CRANES = 2  # Number of quay cranes
NUM_TRUCKS = 3  # Number of terminal trucks
SIMULATION_TIME = 100  # Simulation time in minutes
ARRIVAL_INTERVAL = 5  # Average interval between vessel arrivals
SERVICE_TIME_MIN = 10  # Minimum service time at the berth
SERVICE_TIME_MAX = 20  # Maximum service time at the berth
CONTAINER_MOVE_TIME = 3  # Time for crane to move one container
TRUCK_TRIP_TIME = 6  # Time for truck to drop off a container and return
NUM_CONTAINERS_MIN = 5  # Minimum number of containers per vessel
NUM_CONTAINERS_MAX = 15  # Maximum number of containers per vessel

def vessel(env, name, berths, cranes, trucks):
    """
    Vessel process function.
    Args:
    - env: The SimPy environment.
    - name: The name of the vessel.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{name} arriving at {env.now:.2f}")
    with berths.request() as berth_request:
        yield berth_request
        print(f"{name} starting service at {env.now:.2f}")

        # Request a quay crane
        with cranes.request() as crane_request:
            yield crane_request
            num_containers = random.randint(NUM_CONTAINERS_MIN, NUM_CONTAINERS_MAX)
            print(f"{name} will move {num_containers} containers")

            for i in range(num_containers):
                with trucks.request() as truck_request:
                    yield truck_request
                    print(f"{name} using crane at {env.now:.2f} to move container {i+1}")
                    yield env.timeout(CONTAINER_MOVE_TIME)
                    env.process(truck_trip(env, name, i+1, trucks))

        service_duration = random.randint(SERVICE_TIME_MIN, SERVICE_TIME_MAX)
        yield env.timeout(service_duration)
        print(f"{name} leaving at {env.now:.2f}")

def truck_trip(env, vessel_name, container_num, trucks):
    """
    Truck trip function to simulate transporting a container to the yard block and returning.
    Args:
    - env: The SimPy environment.
    - vessel_name: The name of the vessel.
    - container_num: The container number.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{vessel_name} container {container_num} loaded onto truck at {env.now:.2f}")
    yield env.timeout(TRUCK_TRIP_TIME)
    trucks.release(trucks.request())
    print(f"{vessel_name} container {container_num} dropped off at yard block and truck returned at {env.now:.2f}")

def vessel_generator(env, berths, cranes, trucks):
    """
    Vessel generator function to create new vessels.
    Args:
    - env: The SimPy environment.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    vessel_count = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        vessel_count += 1
        env.process(vessel(env, f"Vessel {vessel_count}", berths, cranes, trucks))

# Create the SimPy environment
env = simpy.Environment()

# Create the berths, cranes, and trucks resources
berths = simpy.Resource(env, NUM_BERTHS)
cranes = simpy.Resource(env, NUM_CRANES)
trucks = simpy.Resource(env, NUM_TRUCKS)

# Start the vessel generator process
env.process(vessel_generator(env, berths, cranes, trucks))

# Run the simulation
env.run(until=SIMULATION_TIME)


Vessel 1 arriving at 1.72
Vessel 1 starting service at 1.72
Vessel 1 will move 15 containers
Vessel 1 using crane at 1.72 to move container 1
Vessel 2 arriving at 4.48
Vessel 2 starting service at 4.48
Vessel 2 will move 14 containers
Vessel 2 using crane at 4.48 to move container 1
Vessel 1 container 1 loaded onto truck at 4.72
Vessel 1 using crane at 4.72 to move container 2
Vessel 2 container 1 loaded onto truck at 7.48
Vessel 2 using crane at 7.48 to move container 2
Vessel 1 container 2 loaded onto truck at 7.72
Vessel 1 using crane at 7.72 to move container 3
Vessel 2 container 2 loaded onto truck at 10.48
Vessel 2 using crane at 10.48 to move container 3
Vessel 1 container 1 dropped off at yard block and truck returned at 10.72
Vessel 1 container 3 loaded onto truck at 10.72
Vessel 1 using crane at 10.72 to move container 4
Vessel 2 container 1 dropped off at yard block and truck returned at 13.48
Vessel 2 container 3 loaded onto truck at 13.48
Vessel 2 using crane at 13.48 to m

5.**Create** a simple log (print statements does the job) for each event, e.g. vessel arriving, vessel berthing, quay crane moves a container, etc. The log should include the current time using SimPy Environment.now (https://simpy.readthedocs.io/en/latest/api_reference/simpy.core.html#simpy.core.Environment.now).

In [3]:

!pip install simpy
import simpy
import random

# Constants
NUM_BERTHS = 2  # Number of berths
NUM_CRANES = 2  # Number of quay cranes
NUM_TRUCKS = 3  # Number of terminal trucks
SIMULATION_TIME = 100  # Simulation time in minutes
ARRIVAL_INTERVAL = 5  # Average interval between vessel arrivals
SERVICE_TIME_MIN = 10  # Minimum service time at the berth
SERVICE_TIME_MAX = 20  # Maximum service time at the berth
CONTAINER_MOVE_TIME = 3  # Time for crane to move one container
TRUCK_TRIP_TIME = 6  # Time for truck to drop off a container and return
NUM_CONTAINERS_MIN = 5  # Minimum number of containers per vessel
NUM_CONTAINERS_MAX = 15  # Maximum number of containers per vessel

def vessel(env, name, berths, cranes, trucks):
    """
    Vessel process function.
    Args:
    - env: The SimPy environment.
    - name: The name of the vessel.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {name} arriving")
    with berths.request() as berth_request:
        yield berth_request
        print(f"{env.now:.2f}: {name} berthing")

        # Request a quay crane
        with cranes.request() as crane_request:
            yield crane_request
            num_containers = random.randint(NUM_CONTAINERS_MIN, NUM_CONTAINERS_MAX)
            print(f"{env.now:.2f}: {name} will move {num_containers} containers")

            for i in range(num_containers):
                with trucks.request() as truck_request:
                    yield truck_request
                    print(f"{env.now:.2f}: {name} using crane to move container {i+1}")
                    yield env.timeout(CONTAINER_MOVE_TIME)
                    env.process(truck_trip(env, name, i+1, trucks))

        service_duration = random.randint(SERVICE_TIME_MIN, SERVICE_TIME_MAX)
        yield env.timeout(service_duration)
        print(f"{env.now:.2f}: {name} leaving")

def truck_trip(env, vessel_name, container_num, trucks):
    """
    Truck trip function to simulate transporting a container to the yard block and returning.
    Args:
    - env: The SimPy environment.
    - vessel_name: The name of the vessel.
    - container_num: The container number.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {vessel_name} container {container_num} loaded onto truck")
    yield env.timeout(TRUCK_TRIP_TIME)
    trucks.release(trucks.request())
    print(f"{env.now:.2f}: {vessel_name} container {container_num} dropped off at yard block and truck returned")

def vessel_generator(env, berths, cranes, trucks):
    """
    Vessel generator function to create new vessels.
    Args:
    - env: The SimPy environment.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    vessel_count = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        vessel_count += 1
        env.process(vessel(env, f"Vessel {vessel_count}", berths, cranes, trucks))

# Create the SimPy environment
env = simpy.Environment()

# Create the berths, cranes, and trucks resources
berths = simpy.Resource(env, NUM_BERTHS)
cranes = simpy.Resource(env, NUM_CRANES)
trucks = simpy.Resource(env, NUM_TRUCKS)

# Start the vessel generator process
env.process(vessel_generator(env, berths, cranes, trucks))

# Run the simulation
env.run(until=SIMULATION_TIME)


2.75: Vessel 1 arriving
2.75: Vessel 1 berthing
2.75: Vessel 1 will move 10 containers
2.75: Vessel 1 using crane to move container 1
3.80: Vessel 2 arriving
3.80: Vessel 2 berthing
3.80: Vessel 2 will move 11 containers
3.80: Vessel 2 using crane to move container 1
5.75: Vessel 1 container 1 loaded onto truck
5.75: Vessel 1 using crane to move container 2
6.35: Vessel 3 arriving
6.67: Vessel 4 arriving
6.80: Vessel 2 container 1 loaded onto truck
6.80: Vessel 2 using crane to move container 2
6.94: Vessel 5 arriving
8.75: Vessel 1 container 2 loaded onto truck
8.75: Vessel 1 using crane to move container 3
9.80: Vessel 2 container 2 loaded onto truck
9.80: Vessel 2 using crane to move container 3
11.75: Vessel 1 container 1 dropped off at yard block and truck returned
11.75: Vessel 1 container 3 loaded onto truck
11.75: Vessel 1 using crane to move container 4
12.80: Vessel 2 container 1 dropped off at yard block and truck returned
12.80: Vessel 2 container 3 loaded onto truck
12.80:

6.	When all containers have been lifted, the vessel leaves and the berth slot is free to be used by the next vessel

In [4]:
!pip install simpy
import simpy
import random

# Constants
NUM_BERTHS = 2  # Number of berths
NUM_CRANES = 2  # Number of quay cranes
NUM_TRUCKS = 3  # Number of terminal trucks
SIMULATION_TIME = 100  # Simulation time in minutes
ARRIVAL_INTERVAL = 5  # Average interval between vessel arrivals
CONTAINER_MOVE_TIME = 3  # Time for crane to move one container
TRUCK_TRIP_TIME = 6  # Time for truck to drop off a container and return
NUM_CONTAINERS_MIN = 5  # Minimum number of containers per vessel
NUM_CONTAINERS_MAX = 15  # Maximum number of containers per vessel

def vessel(env, name, berths, cranes, trucks):
    """
    Vessel process function.
    Args:
    - env: The SimPy environment.
    - name: The name of the vessel.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {name} arriving")
    with berths.request() as berth_request:
        yield berth_request
        print(f"{env.now:.2f}: {name} berthing")

        # Request a quay crane
        with cranes.request() as crane_request:
            yield crane_request
            num_containers = random.randint(NUM_CONTAINERS_MIN, NUM_CONTAINERS_MAX)
            print(f"{env.now:.2f}: {name} will move {num_containers} containers")

            for i in range(num_containers):
                with trucks.request() as truck_request:
                    yield truck_request
                    print(f"{env.now:.2f}: {name} using crane to move container {i+1}")
                    yield env.timeout(CONTAINER_MOVE_TIME)
                    env.process(truck_trip(env, name, i+1, trucks))

        print(f"{env.now:.2f}: {name} all containers moved, leaving berth")
    print(f"{env.now:.2f}: {name} leaving")

def truck_trip(env, vessel_name, container_num, trucks):
    """
    Truck trip function to simulate transporting a container to the yard block and returning.
    Args:
    - env: The SimPy environment.
    - vessel_name: The name of the vessel.
    - container_num: The container number.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {vessel_name} container {container_num} loaded onto truck")
    yield env.timeout(TRUCK_TRIP_TIME)
    trucks.release(trucks.request())
    print(f"{env.now:.2f}: {vessel_name} container {container_num} dropped off at yard block and truck returned")

def vessel_generator(env, berths, cranes, trucks):
    """
    Vessel generator function to create new vessels.
    Args:
    - env: The SimPy environment.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    vessel_count = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        vessel_count += 1
        env.process(vessel(env, f"Vessel {vessel_count}", berths, cranes, trucks))

# Create the SimPy environment
env = simpy.Environment()

# Create the berths, cranes, and trucks resources
berths = simpy.Resource(env, NUM_BERTHS)
cranes = simpy.Resource(env, NUM_CRANES)
trucks = simpy.Resource(env, NUM_TRUCKS)

# Start the vessel generator process
env.process(vessel_generator(env, berths, cranes, trucks))

# Run the simulation
env.run(until=SIMULATION_TIME)


0.89: Vessel 1 arriving
0.89: Vessel 1 berthing
0.89: Vessel 1 will move 12 containers
0.89: Vessel 1 using crane to move container 1
2.44: Vessel 2 arriving
2.44: Vessel 2 berthing
2.44: Vessel 2 will move 13 containers
2.44: Vessel 2 using crane to move container 1
3.89: Vessel 1 container 1 loaded onto truck
3.89: Vessel 1 using crane to move container 2
5.44: Vessel 2 container 1 loaded onto truck
5.44: Vessel 2 using crane to move container 2
6.89: Vessel 1 container 2 loaded onto truck
6.89: Vessel 1 using crane to move container 3
8.44: Vessel 2 container 2 loaded onto truck
8.44: Vessel 2 using crane to move container 3
9.89: Vessel 1 container 1 dropped off at yard block and truck returned
9.89: Vessel 1 container 3 loaded onto truck
9.89: Vessel 1 using crane to move container 4
11.44: Vessel 2 container 1 dropped off at yard block and truck returned
11.44: Vessel 2 container 3 loaded onto truck
11.44: Vessel 2 using crane to move container 4
12.89: Vessel 1 container 2 dropp

7.	Run the simulation for some time using env.run(until=SIMULATION_TIME) where SIMULATION_TIME is a parameter the user can set.

In [5]:
!pip install simpy
import simpy
import random

# Constants
NUM_BERTHS = 2  # Number of berths
NUM_CRANES = 2  # Number of quay cranes
NUM_TRUCKS = 3  # Number of terminal trucks
ARRIVAL_INTERVAL = 5  # Average interval between vessel arrivals
CONTAINER_MOVE_TIME = 3  # Time for crane to move one container
TRUCK_TRIP_TIME = 6  # Time for truck to drop off a container and return
NUM_CONTAINERS_MIN = 5  # Minimum number of containers per vessel
NUM_CONTAINERS_MAX = 15  # Maximum number of containers per vessel

def vessel(env, name, berths, cranes, trucks):
    """
    Vessel process function.
    Args:
    - env: The SimPy environment.
    - name: The name of the vessel.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {name} arriving")
    with berths.request() as berth_request:
        yield berth_request
        print(f"{env.now:.2f}: {name} berthing")

        # Request a quay crane
        with cranes.request() as crane_request:
            yield crane_request
            num_containers = random.randint(NUM_CONTAINERS_MIN, NUM_CONTAINERS_MAX)
            print(f"{env.now:.2f}: {name} will move {num_containers} containers")

            for i in range(num_containers):
                with trucks.request() as truck_request:
                    yield truck_request
                    print(f"{env.now:.2f}: {name} using crane to move container {i+1}")
                    yield env.timeout(CONTAINER_MOVE_TIME)
                    env.process(truck_trip(env, name, i+1, trucks))

        print(f"{env.now:.2f}: {name} all containers moved, leaving berth")
    print(f"{env.now:.2f}: {name} leaving")

def truck_trip(env, vessel_name, container_num, trucks):
    """
    Truck trip function to simulate transporting a container to the yard block and returning.
    Args:
    - env: The SimPy environment.
    - vessel_name: The name of the vessel.
    - container_num: The container number.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {vessel_name} container {container_num} loaded onto truck")
    yield env.timeout(TRUCK_TRIP_TIME)
    trucks.release(trucks.request())
    print(f"{env.now:.2f}: {vessel_name} container {container_num} dropped off at yard block and truck returned")

def vessel_generator(env, berths, cranes, trucks):
    """
    Vessel generator function to create new vessels.
    Args:
    - env: The SimPy environment.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    vessel_count = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        vessel_count += 1
        env.process(vessel(env, f"Vessel {vessel_count}", berths, cranes, trucks))

# Function to run the simulation
def run_simulation(simulation_time):
    # Create the SimPy environment
    env = simpy.Environment()

    # Create the berths, cranes, and trucks resources
    berths = simpy.Resource(env, NUM_BERTHS)
    cranes = simpy.Resource(env, NUM_CRANES)
    trucks = simpy.Resource(env, NUM_TRUCKS)

    # Start the vessel generator process
    env.process(vessel_generator(env, berths, cranes, trucks))

    # Run the simulation
    env.run(until=simulation_time)

# User sets the simulation time
SIMULATION_TIME = 100  # User can set this parameter

# Run the simulation with the specified time
run_simulation(SIMULATION_TIME)


5.65: Vessel 1 arriving
5.65: Vessel 1 berthing
5.65: Vessel 1 will move 9 containers
5.65: Vessel 1 using crane to move container 1
5.87: Vessel 2 arriving
5.87: Vessel 2 berthing
5.87: Vessel 2 will move 13 containers
5.87: Vessel 2 using crane to move container 1
6.08: Vessel 3 arriving
8.65: Vessel 1 container 1 loaded onto truck
8.65: Vessel 1 using crane to move container 2
8.87: Vessel 2 container 1 loaded onto truck
8.87: Vessel 2 using crane to move container 2
11.65: Vessel 1 container 2 loaded onto truck
11.65: Vessel 1 using crane to move container 3
11.87: Vessel 2 container 2 loaded onto truck
11.87: Vessel 2 using crane to move container 3
14.65: Vessel 1 container 1 dropped off at yard block and truck returned
14.65: Vessel 1 container 3 loaded onto truck
14.65: Vessel 1 using crane to move container 4
14.87: Vessel 2 container 1 dropped off at yard block and truck returned
14.87: Vessel 2 container 3 loaded onto truck
14.87: Vessel 2 using crane to move container 4
17.

8 **solution**

In [7]:
!pip install simpy
import simpy
import random

# Constants
NUM_BERTHS = 2  # Number of berths
NUM_CRANES = 2  # Number of quay cranes
NUM_TRUCKS = 3  # Number of terminal trucks
ARRIVAL_INTERVAL = 5  # Average interval between vessel arrivals
CONTAINER_MOVE_TIME = 3  # Time for crane to move one container
TRUCK_TRIP_TIME = 6  # Time for truck to drop off a container and return
NUM_CONTAINERS_MIN = 5  # Minimum number of containers per vessel
NUM_CONTAINERS_MAX = 15  # Maximum number of containers per vessel

def vessel(env, name, berths, cranes, trucks):
    """
    Vessel process function.
    Args:
    - env: The SimPy environment.
    - name: The name of the vessel.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {name} arriving")
    with berths.request() as berth_request:
        yield berth_request
        print(f"{env.now:.2f}: {name} berthing")

        # Request a quay crane
        with cranes.request() as crane_request:
            yield crane_request
            num_containers = random.randint(NUM_CONTAINERS_MIN, NUM_CONTAINERS_MAX)
            print(f"{env.now:.2f}: {name} will move {num_containers} containers")

            for i in range(num_containers):
                with trucks.request() as truck_request:
                    yield truck_request
                    print(f"{env.now:.2f}: {name} using crane to move container {i+1}")
                    yield env.timeout(CONTAINER_MOVE_TIME)
                    env.process(truck_trip(env, name, i+1, trucks))

        print(f"{env.now:.2f}: {name} all containers moved, leaving berth")
    print(f"{env.now:.2f}: {name} leaving")

def truck_trip(env, vessel_name, container_num, trucks):
    """
    Truck trip function to simulate transporting a container to the yard block and returning.
    Args:
    - env: The SimPy environment.
    - vessel_name: The name of the vessel.
    - container_num: The container number.
    - trucks: The resource representing the terminal trucks.
    """
    print(f"{env.now:.2f}: {vessel_name} container {container_num} loaded onto truck")
    yield env.timeout(TRUCK_TRIP_TIME)
    trucks.release(trucks.request())
    print(f"{env.now:.2f}: {vessel_name} container {container_num} dropped off at yard block and truck returned")

def vessel_generator(env, berths, cranes, trucks):
    """
    Vessel generator function to create new vessels.
    Args:
    - env: The SimPy environment.
    - berths: The resource representing the berths.
    - cranes: The resource representing the quay cranes.
    - trucks: The resource representing the terminal trucks.
    """
    vessel_count = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        vessel_count += 1
        env.process(vessel(env, f"Vessel {vessel_count}", berths, cranes, trucks))

# Function to run the simulation
def run_simulation(simulation_time):
    # Create the SimPy environment
    env = simpy.Environment()

    # Create the berths, cranes, and trucks resources
    berths = simpy.Resource(env, NUM_BERTHS)
    cranes = simpy.Resource(env, NUM_CRANES)
    trucks = simpy.Resource(env, NUM_TRUCKS)

    # Start the vessel generator process
    env.process(vessel_generator(env, berths, cranes, trucks))

    # Run the simulation
    env.run(until=simulation_time)

# User sets the simulation time
SIMULATION_TIME = 100  # User can set this parameter

# Run the simulation with the specified time
run_simulation(SIMULATION_TIME)


1.50: Vessel 1 arriving
1.50: Vessel 1 berthing
1.50: Vessel 1 will move 9 containers
1.50: Vessel 1 using crane to move container 1
4.50: Vessel 1 container 1 loaded onto truck
4.50: Vessel 1 using crane to move container 2
7.50: Vessel 1 container 2 loaded onto truck
7.50: Vessel 1 using crane to move container 3
10.27: Vessel 2 arriving
10.27: Vessel 2 berthing
10.27: Vessel 2 will move 7 containers
10.27: Vessel 2 using crane to move container 1
10.32: Vessel 3 arriving
10.50: Vessel 1 container 1 dropped off at yard block and truck returned
10.50: Vessel 1 container 3 loaded onto truck
10.50: Vessel 1 using crane to move container 4
13.17: Vessel 4 arriving
13.27: Vessel 2 container 1 loaded onto truck
13.27: Vessel 2 using crane to move container 2
13.50: Vessel 1 container 2 dropped off at yard block and truck returned
13.50: Vessel 1 container 4 loaded onto truck
13.50: Vessel 1 using crane to move container 5
16.27: Vessel 2 container 2 loaded onto truck
16.27: Vessel 2 using 